In [1]:
%%javascript
IPython.notebook.clear_all_output();

<IPython.core.display.Javascript object>

In [2]:
import csv
import pandas as pd
import numpy as np

In [ ]:
#all_gaze
def allgaze(participants,user_start,gazer_start):
    user = user_start
    gazer = gazer_start
    for p in participants:
        ptp = p
        data=pd.read_csv("../Data/Experiment Data/Eye Tracking/"+ "User "+str(gazer)+"_fixations.csv")
        time_data=data.columns[3][5:-1]
        #display(time_data)
        year,month,day=[int(x) for x in time_data.split(' ')[0].split('/')]
        hour,min,sec,mili= [int(x) for x  in time_data.split(' ')[1].replace(".", ":").split(':')]
        from datetime import datetime, timedelta
        start_time= datetime(year,month,day,hour,min,sec)
        timestamps=[]
        for i in range(len(data)):
            timestamps.append((start_time+timedelta( microseconds=data.iloc[i][3]*10**6)).strftime("%H:%M:%S"))
        
        data['timestamps']=timestamps

        path_analysis = "../Data/Experiment Analysis/" + ptp + "/"
        data.to_csv(path_analysis+"fixtations_"+ptp+".csv", index = False)
        gazer = gazer + 1
        user = user + 1
        print(str(p) + " all_gaze Done!")

In [ ]:
#fixations
def fixations(participants,user_start,gazer_start):
    user = user_start
    gazer = gazer_start
    for p in participants:
        ptp = p
        data=pd.read_csv("../Data/Experiment Data/" + ptp +"/"+"User"+str(user)+"_EyeTracking/"+ "User "+str(gazer)+"_fixations.csv")
        time_data=data.columns[3][5:-1]
        year,month,day=[int(x) for x in time_data.split(' ')[0].split('/')]
        hour,min,sec,mili= [int(x) for x  in time_data.split(' ')[1].replace(".", ":").split(':')]
        from datetime import datetime, timedelta
        start_time= datetime(year,month,day,hour,min,sec)
        timestamps=[]
        for i in range(len(data)):
            timestamps.append((start_time+timedelta( microseconds=data.iloc[i][3]*10**6)).strftime("%H:%M:%S"))
        
        data['timestamps']=timestamps

        path_analysis = "../Data/Experiment Analysis/" + ptp + "/"
        data.to_csv(path_analysis+"Fixations_"+ptp+".csv", index = False)
        gazer = gazer + 1
        user = user + 1
        print(str(p) + " fixations Done!")

In [ ]:
#participants = ["2001_0","2002_0","2003_0","2004_0","2005_0","2006_0","2007_0","2008_0","2009_0","2010_0","2011_0","2012_0","2013_0","2014_0"]
#participants = ["2001_0","2002","2003_0","2004_0","2005_0","2006_0","2007_0","2009_0","2010_0","2011_0","2013_0","2014_0","2016_0","2017_0","2019_0","2020_0","2021_0","2022_0","2023_0","2024_0","2025_0"]
gazer_start = 25

user_start = 2000 + gazer_start
participants = [str(user_start)+"_0"]

#allgaze(participants,user_start,gazer_start)
print("All gaze done!")
fixations(participants,user_start,gazer_start)
print("Fixations done!")

In [ ]:
from datetime import datetime
import time

def datamanipulation(participant):
    ptp = participant
    path_data = "../Data/Experiment Data/" + ptp +"/"
    path_analysis = "../Data/Experiment Analysis/" + ptp + "/"

    #path_data = "../Data/Experiment Analysis/" + ptp + "/" +"Fixations_"+ptp+".csv"

    responsData = pd.read_csv(path_data + "Responses.csv")
    experimentData = pd.read_csv ("../Data/Experiment Analysis/" + ptp + "/" +"fixations_"+ptp+".csv")
    #display(responsData)
    display(experimentData)
    
    #display(responsData.StartTime[0:1])
    start_time = pd.to_datetime(responsData.StartTime[0:1],format='%H:%M:%S').dt.time
    end_time = pd.to_datetime(responsData.ResponseTime[0:1],format='%H:%M:%S').dt.time
    #display(start_time)
    #display(end_time)
    clock_time = pd.to_datetime(experimentData["timestamps"],format='%H:%M:%S').dt.time
    #display(clock_time)
    frame = []
    frameAvg = []
    frameAvgRest = []

    framefpr = []
    #framefprRest = []
    for index, row in responsData.iterrows():
        Id,CelebrityName,VideoType,Response,ParticipantId,ResponseTime,VideoId,StartTime,FamId,SeenBefore,TimeSpent,Confusion,Correction = row

        start_time = pd.to_datetime(responsData.StartTime[index:index+1],format='%H:%M:%S').dt.time
        end_time = pd.to_datetime(responsData.ResponseTime[index:index+1],format='%H:%M:%S').dt.time

        mask = (clock_time < end_time[index]) & (clock_time >= start_time[index]) 

        data = experimentData[mask]    
        data["ResponseId"] = Id
        data["CelebrityName"] = CelebrityName
        data["VideoType"] = VideoType
        data["Response"] = Response
        data["ParticipantId"] = ParticipantId
        data["ResponseTime"] = ResponseTime
        data["VideoId"] = VideoId
        data["StartTime"] = StartTime
        data["FamiliarizationId"] = FamId
        data["TimeSpent"] = TimeSpent
        data["Confusion"] = Confusion
        data["Correction"] = Correction
        #data["SeenBefore"] == seenBefore
        count = 0
        #display(data)
        if not (data.empty):
            frame.append(data)
    result = pd.concat(frame)
    #display(result)    
    result.to_csv(path_analysis+"Fixtation_masked"+ptp+".csv", index = False)
    print(ptp + " DONE!")

In [ ]:
#participants = ["2001_0","2002_0","2003_0","2004_0","2005_0","2006_0","2007_0","2008_0","2009_0","2010_0","2011_0","2012_0","2013_0","2014_0"]
participants = ["2018_0"]
#participants = ["2003_0","2004_0","2005_0","2006_0","2007_0","2009_0","2010_0","2011_0","2013_0","2014_0","2016_0","2017_0","2019_0","2020_0","2021_0","2022_0","2023_0","2024_0","2025_0"]

for p in participants:
    datamanipulation(p)

In [ ]:
# Real vs Fake fixations
frame_fake_alike = []
frame_real_alike = []
frame_fake_fam = []
frame_real_fam = []
frame_fake_bfam = []
frame_real_bfam = []

fake_frame = {"VideoId":[],"ParticipantId":[],"VideoType":[],"CelebrityName":[],"Confusion":[],"Correction":[],"FPOGX":[],"FPOGY":[],"FPOGS":[],"FPOGD":[],"FPOGID":[],"FPOGV":[],"AOI":[],"SACCADE_MAG":[],"SACCADE_DIR":[]}
real_frame = {"VideoId":[],"ParticipantId":[],"VideoType":[],"CelebrityName":[],"Confusion":[],"Correction":[],"FPOGX":[],"FPOGY":[],"FPOGS":[],"FPOGD":[],"FPOGID":[],"FPOGV":[],"AOI":[],"SACCADE_MAG":[],"SACCADE_DIR":[]}
avg_fake = {"VideoId":[],"ParticipantId":[],"VideoType":[],"CelebrityName":[],"Confusion":[],"Correction":[],"FPOGX":[],"FPOGY":[],"FPOGS":[],"FPOGD":[],"FPOGID":[],"FPOGV":[],"AOI":[],"SACCADE_MAG":[],"SACCADE_DIR":[]}
avg_real = {"VideoId":[],"ParticipantId":[],"VideoType":[],"CelebrityName":[],"Confusion":[],"Correction":[],"FPOGX":[],"FPOGY":[],"FPOGS":[],"FPOGD":[],"FPOGID":[],"FPOGV":[],"AOI":[],"SACCADE_MAG":[],"SACCADE_DIR":[]}

FakeData_ALIKE = pd.DataFrame(fake_frame)
RealData_ALIKE = pd.DataFrame(real_frame)
Avg_Fake_Data_ALIKE = pd.DataFrame(avg_fake)
Avg_Real_Data_ALIKE = pd.DataFrame(avg_real)

FakeData_FAM = pd.DataFrame(fake_frame)
RealData_FAM = pd.DataFrame(real_frame)
Avg_Fake_Data_FAM = pd.DataFrame(avg_fake)
Avg_Real_Data_FAM = pd.DataFrame(avg_real)

FakeData_BFAM = pd.DataFrame(fake_frame)
RealData_BFAM = pd.DataFrame(real_frame)
Avg_Fake_Data_BFAM = pd.DataFrame(avg_fake)
Avg_Real_Data_BFAM = pd.DataFrame(avg_real)


participants = ["2001_0","2002_0","2003_0","2004_0","2005_0","2006_0","2007_0","2008_0","2009_0","2010_0","2011_0","2012_0","2013_0","2014_0","2016_0","2017_0","2018_0","2019_0","2020_0","2021_0","2022_0","2023_0","2024_0","2025_0"]
#participants = ["2001_0","2002_0"]
for p in participants:
    file_path = "Fixtation_masked"+p+".csv"
    path_analysis = "../Data/Experiment Analysis/" + p + "/" + file_path
    Fixation_Data =  pd.read_csv(path_analysis)
    #display(Fixation_Data)
    for index, row in Fixation_Data.iterrows():
        if row["VideoType"] == "ALIKE_FAKE":
            FakeData_ALIKE.loc[len(FakeData_ALIKE.index)] = row[["ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]] 
        elif row["VideoType"] == "CELEBRITY_FAKE":
            FakeData_FAM.loc[len(FakeData_FAM.index)] = row[["ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]] 
        
        elif  row["VideoType"] == "NONCELEBRITY_FAKE":
            FakeData_BFAM.loc[len(FakeData_BFAM.index)] = row[["ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]] 
        
        elif row["VideoType"] == "ALIKE_REAL":
            RealData_ALIKE.loc[len(RealData_ALIKE.index)] = row[["ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]] 
        elif row["VideoType"] == "CELEBRITY_REAL":
            RealData_FAM.loc[len(RealData_FAM.index)] = row[["ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]] 
            
        elif  row["VideoType"] == "NONCELEBRITY_REAL":
            RealData_BFAM.loc[len(RealData_BFAM.index)] = row[["ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]] 
    
    Avg_Fake_Data_ALIKE.loc["AVG"] = FakeData_ALIKE[["ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]].mean(axis=0,numeric_only=True).round(3)
    Avg_Real_Data_ALIKE.loc["AVG"] = RealData_ALIKE[["ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]].mean(axis=0,numeric_only=True).round(3)
    Avg_Fake_Data_FAM.loc["AVG"] = FakeData_FAM[["ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]].mean(axis=0,numeric_only=True).round(3)
    Avg_Real_Data_FAM.loc["AVG"] = RealData_FAM[["ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]].mean(axis=0,numeric_only=True).round(3)
    Avg_Fake_Data_BFAM.loc["AVG"] = FakeData_BFAM[["ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]].mean(axis=0,numeric_only=True).round(3)
    Avg_Real_Data_BFAM.loc["AVG"] = RealData_BFAM[["ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]].mean(axis=0,numeric_only=True).round(3)

    print(p+ " is done!")
    frame_fake_alike.append(Avg_Fake_Data_ALIKE.tail())
    frame_real_alike.append(Avg_Real_Data_ALIKE.tail())
    frame_fake_fam.append(Avg_Fake_Data_FAM.tail())
    frame_real_fam.append(Avg_Real_Data_FAM.tail())
    frame_fake_bfam.append(Avg_Fake_Data_BFAM.tail())
    frame_real_bfam.append(Avg_Real_Data_BFAM.tail())
    
    FakeData_ALIKE.drop(FakeData_ALIKE.index, inplace=True)
    RealData_ALIKE.drop(RealData_ALIKE.index, inplace=True)
    FakeData_FAM.drop(FakeData_FAM.index, inplace=True)
    RealData_FAM.drop(RealData_FAM.index, inplace=True)
    FakeData_BFAM.drop(FakeData_BFAM.index, inplace=True)
    RealData_BFAM.drop(RealData_BFAM.index, inplace=True)
    
    
    
    Avg_Fake_Data_ALIKE.drop(Avg_Fake_Data_ALIKE.index, inplace=True)
    Avg_Real_Data_ALIKE.drop(Avg_Real_Data_ALIKE.index, inplace=True)
    Avg_Fake_Data_FAM.drop(Avg_Fake_Data_FAM.index, inplace=True)
    Avg_Real_Data_FAM.drop(Avg_Real_Data_FAM.index, inplace=True)
    Avg_Fake_Data_BFAM.drop(Avg_Fake_Data_BFAM.index, inplace=True)
    Avg_Real_Data_BFAM.drop(Avg_Real_Data_BFAM.index, inplace=True)
    
    
#display(frame1)
#display(frame2)

AVG_fake_ALIKE = pd.concat(frame_fake_alike)
AVG_real_ALIKE = pd.concat(frame_real_alike)
AVG_fake_FAM = pd.concat(frame_fake_fam)
AVG_real_FAM = pd.concat(frame_real_fam)
AVG_fake_BFAM = pd.concat(frame_fake_bfam)
AVG_real_BFAM = pd.concat(frame_real_bfam)
#display(AVG_fake)
#display(AVG_real)
    #Avg_Fake_Data = Avg_Fake_Data.append(FakeData.tail(1), ignore_index=True)
    #Avg_Real_Data = Avg_Real_Data.append(RealData.tail(1), ignore_index=True)


#display(Avg_Fake_Data)
#display(Avg_Real_Data)
AVG_fake_ALIKE.to_csv("../Data/Experiment Analysis/"+"ALIKE_AVG_Fake_Fixtations.csv", index = False)
AVG_real_ALIKE.to_csv("../Data/Experiment Analysis/"+"ALIKE_AVG_Real_Fixtations.csv", index = False)
AVG_fake_FAM.to_csv("../Data/Experiment Analysis/"+"FAM_AVG_Fake_Fixtations.csv", index = False)
AVG_real_FAM.to_csv("../Data/Experiment Analysis/"+"FAM_AVG_Real_Fixtations.csv", index = False)
AVG_fake_BFAM.to_csv("../Data/Experiment Analysis/"+"BFAM_AVG_Fake_Fixtations.csv", index = False)
AVG_real_BFAM.to_csv("../Data/Experiment Analysis/"+"BFAM_AVG_Real_Fixtations.csv", index = False)

In [ ]:
# Real vs Fake fixations
frame_fake_alike = []
frame_real_alike = []
frame_fake_fam = []
frame_real_fam = []
frame_fake_bfam = []
frame_real_bfam = []

fake_frame = {"ParticipantId":[],"VideoType":[],"CelebrityName":[],"Confusion":[],"Correction":[],"FPOGX":[],"FPOGY":[],"FPOGS":[],"FPOGD":[],"FPOGID":[],"FPOGV":[],"AOI":[],"SACCADE_MAG":[],"SACCADE_DIR":[]}
real_frame = {"ParticipantId":[],"VideoType":[],"CelebrityName":[],"Confusion":[],"Correction":[],"FPOGX":[],"FPOGY":[],"FPOGS":[],"FPOGD":[],"FPOGID":[],"FPOGV":[],"AOI":[],"SACCADE_MAG":[],"SACCADE_DIR":[]}
avg_fake = {"ParticipantId":[],"VideoType":[],"CelebrityName":[],"Confusion":[],"Correction":[],"FPOGX":[],"FPOGY":[],"FPOGS":[],"FPOGD":[],"FPOGID":[],"FPOGV":[],"AOI":[],"SACCADE_MAG":[],"SACCADE_DIR":[]}
avg_real = {"ParticipantId":[],"VideoType":[],"CelebrityName":[],"Confusion":[],"Correction":[],"FPOGX":[],"FPOGY":[],"FPOGS":[],"FPOGD":[],"FPOGID":[],"FPOGV":[],"AOI":[],"SACCADE_MAG":[],"SACCADE_DIR":[]}

FakeData_ALIKE = pd.DataFrame(fake_frame)
RealData_ALIKE = pd.DataFrame(real_frame)
Avg_Fake_Data_ALIKE = pd.DataFrame(avg_fake)
Avg_Real_Data_ALIKE = pd.DataFrame(avg_real)

FakeData_FAM = pd.DataFrame(fake_frame)
RealData_FAM = pd.DataFrame(real_frame)
Avg_Fake_Data_FAM = pd.DataFrame(avg_fake)
Avg_Real_Data_FAM = pd.DataFrame(avg_real)

FakeData_BFAM = pd.DataFrame(fake_frame)
RealData_BFAM = pd.DataFrame(real_frame)
Avg_Fake_Data_BFAM = pd.DataFrame(avg_fake)
Avg_Real_Data_BFAM = pd.DataFrame(avg_real)


participants = ["2001_0","2002_0","2003_0","2004_0","2005_0","2006_0","2007_0","2008_0","2009_0","2010_0","2011_0","2012_0","2013_0","2014_0","2016_0","2017_0","2018_0","2019_0","2020_0","2021_0","2022_0","2023_0","2024_0","2025_0"]
#participants = ["2001_0","2002_0"]
for p in participants:
    file_path = "Fixtation_masked"+p+".csv"
    path_analysis = "../Data/Experiment Analysis/" + p + "/" + file_path
    Fixation_Data =  pd.read_csv(path_analysis)
    #display(Fixation_Data)
    for index, row in Fixation_Data.iterrows():
        if row["VideoType"] == "ALIKE_FAKE":
            FakeData_ALIKE.loc[len(FakeData_ALIKE.index)] = row[["ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]]
            fakeasfake_ALIKE = FakeData_ALIKE[FakeData_ALIKE["Confusion"] == "FAKE AS FAKE"]
            fakeasreal_ALIKE = FakeData_ALIKE[FakeData_ALIKE["Confusion"] == "FAKE AS REAL"]
            
            
        elif row["VideoType"] == "CELEBRITY_FAKE":
            FakeData_FAM.loc[len(FakeData_FAM.index)] = row[["ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]] 
            fakeasfake_FAM = FakeData_FAM[FakeData_FAM["Confusion"] == "FAKE AS FAKE"]
            fakeasreal_FAM = FakeData_FAM[FakeData_FAM["Confusion"] == "FAKE AS REAL"]
            
        elif  row["VideoType"] == "NONCELEBRITY_FAKE":
            FakeData_BFAM.loc[len(FakeData_BFAM.index)] = row[["ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]] 
            fakeasfake_BFAM = FakeData_BFAM[FakeData_BFAM["Confusion"] == "FAKE AS FAKE"]
            fakeasreal_BFAM = FakeData_BFAM[FakeData_BFAM["Confusion"] == "FAKE AS REAL"]
                                          
        elif row["VideoType"] == "ALIKE_REAL":
            RealData_ALIKE.loc[len(RealData_ALIKE.index)] = row[["ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]] 
            realasreal_ALIKE = RealData_ALIKE[RealData_ALIKE["Confusion"] == "REAL AS REAL"]
            realasfake_ALIKE = RealData_ALIKE[RealData_ALIKE["Confusion"] == "REAL AS FAKE"]
        elif row["VideoType"] == "CELEBRITY_REAL":
            RealData_FAM.loc[len(RealData_FAM.index)] = row[["ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]] 
            realasreal_FAM = RealData_FAM[RealData_FAM["Confusion"] == "REAL AS REAL"]
            realasfake_FAM = RealData_FAM[RealData_FAM["Confusion"] == "REAL AS FAKE"]
        elif  row["VideoType"] == "NONCELEBRITY_REAL":
            RealData_BFAM.loc[len(RealData_BFAM.index)] = row[["ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]] 
            realasreal_BFAM = RealData_BFAM[RealData_BFAM["Confusion"] == "REAL AS REAL"]
            realasfake_BFAM = RealData_BFAM[RealData_BFAM["Confusion"] == "REAL AS FAKE"]
    print(p+ " is done!")
       
    
fakeasfake_ALIKE.to_csv("../Data/Experiment Analysis/"+"ALIKE_Fake_as_Fake_Fixations.csv", index = False)
fakeasreal_ALIKE.to_csv("../Data/Experiment Analysis/"+"ALIKE_Fake_as_Real_Fixations.csv", index = False)

realasreal_ALIKE.to_csv("../Data/Experiment Analysis/"+"ALIKE_Real_as_Real_Fixations.csv", index = False)
realasfake_ALIKE.to_csv("../Data/Experiment Analysis/"+"ALIKE_Real_as_Fake_Fixations.csv", index = False)

fakeasfake_FAM.to_csv("../Data/Experiment Analysis/"+"FAM_Fake_as_Fake_Fixations.csv", index = False)
fakeasreal_FAM.to_csv("../Data/Experiment Analysis/"+"FAM_Fake_as_Real_Fixations.csv", index = False)

realasreal_FAM.to_csv("../Data/Experiment Analysis/"+"FAM_Real_as_Real_Fixations.csv", index = False)
realasfake_FAM.to_csv("../Data/Experiment Analysis/"+"FAM_Real_as_Fake_Fixations.csv", index = False)

fakeasfake_BFAM.to_csv("../Data/Experiment Analysis/"+"BFAM_Fake_as_Fake_Fixations.csv", index = False)
fakeasreal_BFAM.to_csv("../Data/Experiment Analysis/"+"BFAM_Fake_as_Real_Fixations.csv", index = False)

realasreal_BFAM.to_csv("../Data/Experiment Analysis/"+"BFAM_Real_as_Real_Fixations.csv", index = False)
realasfake_BFAM.to_csv("../Data/Experiment Analysis/"+"BFAM_Real_as_Fake_Fixations.csv", index = False)

#FakeData_BFAM.to_csv("../Data/Experiment Analysis/"+"BFAM_Fake_Fixtations.csv", index = False)
#RealData_ALIKE.to_csv("../Data/Experiment Analysis/"+"ALIKE_Real_Fixtations.csv", index = False)
#RealData_FAM.to_csv("../Data/Experiment Analysis/"+"FAM_Real_Fixtations.csv", index = False)
#RealData_BFAM.to_csv("../Data/Experiment Analysis/"+"BFAM_Real_Fixtations.csv", index = False)

In [ ]:
# Real vs Fake fixations
frame_fake_alike = []
frame_real_alike = []
frame_fake_fam = []
frame_real_fam = []
frame_fake_bfam = []
frame_real_bfam = []

fake_frame = {"Response":[],"ParticipantId":[],"VideoType":[],"CelebrityName":[],"Confusion":[],"Correction":[],"FPOGX":[],"FPOGY":[],"FPOGS":[],"FPOGD":[],"FPOGID":[],"FPOGV":[],"AOI":[],"SACCADE_MAG":[],"SACCADE_DIR":[]}
real_frame = {"Response":[],"ParticipantId":[],"VideoType":[],"CelebrityName":[],"Confusion":[],"Correction":[],"FPOGX":[],"FPOGY":[],"FPOGS":[],"FPOGD":[],"FPOGID":[],"FPOGV":[],"AOI":[],"SACCADE_MAG":[],"SACCADE_DIR":[]}
avg_fake = {"Response":[],"ParticipantId":[],"VideoType":[],"CelebrityName":[],"Confusion":[],"Correction":[],"FPOGX":[],"FPOGY":[],"FPOGS":[],"FPOGD":[],"FPOGID":[],"FPOGV":[],"AOI":[],"SACCADE_MAG":[],"SACCADE_DIR":[]}
avg_real = {"Response":[],"ParticipantId":[],"VideoType":[],"CelebrityName":[],"Confusion":[],"Correction":[],"FPOGX":[],"FPOGY":[],"FPOGS":[],"FPOGD":[],"FPOGID":[],"FPOGV":[],"AOI":[],"SACCADE_MAG":[],"SACCADE_DIR":[]}

FakeData_ALIKE = pd.DataFrame(fake_frame)
RealData_ALIKE = pd.DataFrame(real_frame)
Avg_Fake_Data_ALIKE = pd.DataFrame(avg_fake)
Avg_Real_Data_ALIKE = pd.DataFrame(avg_real)

FakeData_FAM = pd.DataFrame(fake_frame)
RealData_FAM = pd.DataFrame(real_frame)
Avg_Fake_Data_FAM = pd.DataFrame(avg_fake)
Avg_Real_Data_FAM = pd.DataFrame(avg_real)

FakeData_BFAM = pd.DataFrame(fake_frame)
RealData_BFAM = pd.DataFrame(real_frame)
Avg_Fake_Data_BFAM = pd.DataFrame(avg_fake)
Avg_Real_Data_BFAM = pd.DataFrame(avg_real)


participants = ["2001_0","2002_0","2003_0","2004_0","2005_0","2006_0","2007_0","2008_0","2009_0","2010_0","2011_0","2012_0","2013_0","2014_0","2016_0","2017_0","2018_0","2019_0","2020_0","2021_0","2022_0","2023_0","2024_0","2025_0"]
#participants = ["2001_0","2002_0"]
for p in participants:
    file_path = "Fixtation_masked"+p+".csv"
    path_analysis = "../Data/Experiment Analysis/" + p + "/" + file_path
    Fixation_Data =  pd.read_csv(path_analysis)
    #display(Fixation_Data)
    for index, row in Fixation_Data.iterrows():
        if row["Correction"] == 0:
            FakeData_ALIKE.loc[len(FakeData_ALIKE.index)] = row[["Response","ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]] 
        elif row["Correction"] == 1:
            FakeData_FAM.loc[len(FakeData_FAM.index)] = row[["Response","ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]] 
        
        #elif  row["Confusion"] == "REAL AS FAKE":
        #    FakeData_BFAM.loc[len(FakeData_BFAM.index)] = row[["ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]] 
        
        #elif row["Confusion"] == "REAL AS REAL":
        #    RealData_ALIKE.loc[len(RealData_ALIKE.index)] = row[["ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]] 
        
    Avg_Fake_Data_ALIKE.loc["AVG"] = FakeData_ALIKE[["ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]].mean(axis=0,numeric_only=True).round(3)
    #Avg_Real_Data_ALIKE.loc["AVG"] = RealData_ALIKE[["ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]].mean(axis=0,numeric_only=True).round(3)
    Avg_Fake_Data_FAM.loc["AVG"] = FakeData_FAM[["ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]].mean(axis=0,numeric_only=True).round(3)
    #Avg_Real_Data_FAM.loc["AVG"] = RealData_FAM[["ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]].mean(axis=0,numeric_only=True).round(3)
    #Avg_Fake_Data_BFAM.loc["AVG"] = FakeData_BFAM[["ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]].mean(axis=0,numeric_only=True).round(3)
    #Avg_Real_Data_BFAM.loc["AVG"] = RealData_BFAM[["ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]].mean(axis=0,numeric_only=True).round(3)

    print(p+ " is done!")
    frame_fake_alike.append(Avg_Fake_Data_ALIKE.tail())
    #frame_real_alike.append(Avg_Real_Data_ALIKE.tail())
    frame_fake_fam.append(Avg_Fake_Data_FAM.tail())
    #frame_real_fam.append(Avg_Real_Data_FAM.tail())
    #frame_fake_bfam.append(Avg_Fake_Data_BFAM.tail())
    #frame_real_bfam.append(Avg_Real_Data_BFAM.tail())
    
    FakeData_ALIKE.drop(FakeData_ALIKE.index, inplace=True)
    #RealData_ALIKE.drop(RealData_ALIKE.index, inplace=True)
    FakeData_FAM.drop(FakeData_FAM.index, inplace=True)
    #RealData_FAM.drop(RealData_FAM.index, inplace=True)
    #FakeData_BFAM.drop(FakeData_BFAM.index, inplace=True)
    #RealData_BFAM.drop(RealData_BFAM.index, inplace=True)
    
    
    
    Avg_Fake_Data_ALIKE.drop(Avg_Fake_Data_ALIKE.index, inplace=True)
    #Avg_Real_Data_ALIKE.drop(Avg_Real_Data_ALIKE.index, inplace=True)
    Avg_Fake_Data_FAM.drop(Avg_Fake_Data_FAM.index, inplace=True)
    #Avg_Real_Data_FAM.drop(Avg_Real_Data_FAM.index, inplace=True)
    #Avg_Fake_Data_BFAM.drop(Avg_Fake_Data_BFAM.index, inplace=True)
    #Avg_Real_Data_BFAM.drop(Avg_Real_Data_BFAM.index, inplace=True)
    
    
#display(frame1)
#display(frame2)

AVG_fake_ALIKE = pd.concat(frame_fake_alike)
#AVG_real_ALIKE = pd.concat(frame_real_alike)
AVG_fake_FAM = pd.concat(frame_fake_fam)
#AVG_real_FAM = pd.concat(frame_real_fam)
#AVG_fake_BFAM = pd.concat(frame_fake_bfam)
#AVG_real_BFAM = pd.concat(frame_real_bfam)
#display(AVG_fake)
#display(AVG_real)
    #Avg_Fake_Data = Avg_Fake_Data.append(FakeData.tail(1), ignore_index=True)
    #Avg_Real_Data = Avg_Real_Data.append(RealData.tail(1), ignore_index=True)


#display(Avg_Fake_Data)
#display(Avg_Real_Data)
AVG_fake_ALIKE.to_csv("../Data/Experiment Analysis/"+"False_Fixations.csv", index = False)
#AVG_real_ALIKE.to_csv("../Data/Experiment Analysis/"+"RealAsReal_Fixations.csv", index = False)
AVG_fake_FAM.to_csv("../Data/Experiment Analysis/"+"True_Fixations.csv", index = False)
AVG_real_FAM.to_csv("../Data/Experiment Analysis/"+"FAM_AVG_Real_Fixtations.csv", index = False)
AVG_fake_BFAM.to_csv("../Data/Experiment Analysis/"+"RealAsFake_Fixations.csv", index = False)
AVG_real_BFAM.to_csv("../Data/Experiment Analysis/"+"BFAM_AVG_Real_Fixtations.csv", index = False)

In [12]:
# Real vs Fake fixations
frame_fake_alike = []
frame_real_alike = []
frame_fake_fam = []
frame_real_fam = []
frame_fake_bfam = []
frame_real_bfam = []

fake_frame = {"Response":[],"VideoId":[],"ParticipantId":[],"VideoType":[],"CelebrityName":[],"Confusion":[],"Correction":[],"FPOGX":[],"FPOGY":[],"FPOGS":[],"FPOGD":[],"FPOGID":[],"FPOGV":[],"AOI":[],"SACCADE_MAG":[],"SACCADE_DIR":[]}
real_frame = {"ParticipantId":[],"VideoType":[],"CelebrityName":[],"Confusion":[],"Correction":[],"FPOGX":[],"FPOGY":[],"FPOGS":[],"FPOGD":[],"FPOGID":[],"FPOGV":[],"AOI":[],"SACCADE_MAG":[],"SACCADE_DIR":[]}
avg_fake = {"Response":[],"VideoId":[],"ParticipantId":[],"VideoType":[],"CelebrityName":[],"Confusion":[],"Correction":[],"FPOGX":[],"FPOGY":[],"FPOGS":[],"FPOGD":[],"FPOGID":[],"FPOGV":[],"AOI":[],"SACCADE_MAG":[],"SACCADE_DIR":[]}
#avg_real = {"ParticipantId":[],"VideoType":[],"CelebrityName":[],"Confusion":[],"Correction":[],"FPOGX":[],"FPOGY":[],"FPOGS":[],"FPOGD":[],"FPOGID":[],"FPOGV":[],"AOI":[],"SACCADE_MAG":[],"SACCADE_DIR":[]}

FakeData_ALIKE = pd.DataFrame(fake_frame)
RealData_ALIKE = pd.DataFrame(real_frame)
Avg_Fake_Data_ALIKE = pd.DataFrame(avg_fake)
#Avg_Real_Data_ALIKE = pd.DataFrame(avg_real)

FakeData_FAM = pd.DataFrame(fake_frame)
#RealData_FAM = pd.DataFrame(real_frame)
#Avg_Fake_Data_FAM = pd.DataFrame(avg_fake)

#Avg_Real_Data_FAM = pd.DataFrame(avg_real)

#FakeData_BFAM = pd.DataFrame(fake_frame)
#RealData_BFAM = pd.DataFrame(real_frame)
#Avg_Fake_Data_BFAM = pd.DataFrame(avg_fake)
#Avg_Real_Data_BFAM = pd.DataFrame(avg_real)


#participants = ["2001_0","2002_0","2003_0","2004_0","2005_0","2006_0","2007_0","2008_0","2009_0","2010_0","2011_0","2012_0","2013_0","2014_0","2016_0","2017_0","2018_0","2019_0","2020_0","2021_0","2022_0","2023_0","2024_0","2025_0"]
participants = ["2001_0"]
for p in participants:
    file_path = "Fixtation_masked"+p+".csv"
    path_analysis = "../Data/Experiment Analysis/" + p + "/" + file_path
    Fixation_Data =  pd.read_csv(path_analysis)
    #display(Fixation_Data)
    for index, row in Fixation_Data.iterrows():        
        FakeData_ALIKE.loc[len(FakeData_ALIKE.index)] = row[["Response","VideoId","ParticipantId","VideoType","CelebrityName","Confusion","Correction","FPOGX","FPOGY","FPOGS","FPOGD","FPOGID","FPOGV","AOI","SACCADE_MAG","SACCADE_DIR"]]
    RealData_ALIKE = (FakeData_ALIKE.loc[FakeData_ALIKE['Response'] == "Yes"])
    display(RealData_ALIKE)
    
    #RealData_ALIKE.groupby('VideoId','ParticipantId' as_index=False)["VideoId","ParticipantId","FPOGD","SACCADE_MAG","SACCADE_DIR"].mean()
    #FakeData_FAM = FakeData_ALIKE.loc[FaFakeData_ALIKE['Response'] == "No"].groupby('VideoId','ParticipantId' as_index=False)["VideoId","ParticipantId","FPOGD","SACCADE_MAG","SACCADE_DIR"].mean()
    display(RealData_ALIKE)
    display(FakeData_FAM)
    print(p+ " is done!")
       
    
#FakeData_ALIKE.to_csv("../Data/Experiment Analysis/"+"Yes_Response_AVG_Fixations.csv", index = False)
#Avg_Fake_Data_ALIKE.to_csv("../Data/Experiment Analysis/"+"AVG_Response_Fixations.csv", index = False)

,Response,VideoId,ParticipantId,VideoType,CelebrityName,Confusion,Correction,FPOGX,FPOGY,FPOGS,FPOGD,FPOGID,FPOGV,AOI,SACCADE_MAG,SACCADE_DIR
0,Yes,237.0,11.0,CELEBRITY_FAKE,Donald Trump,FAKE AS REAL,0.0,0.50185,0.28159,162.27788,0.29126,313.0,1.0,FAM_FAKE_EYES,451.72058,96.14632
1,Yes,237.0,11.0,CELEBRITY_FAKE,Donald Trump,FAKE AS REAL,0.0,0.51672,0.38556,162.58522,1.95715,314.0,1.0,FAM_FAKE_EYES-FAM_FAKE_MOUTH,127.98897,282.88937
2,Yes,237.0,11.0,CELEBRITY_FAKE,Donald Trump,FAKE AS REAL,0.0,0.57352,0.41739,164.55856,0.46890,315.0,1.0,NaN,115.55151,340.69760
3,Yes,237.0,11.0,CELEBRITY_FAKE,Donald Trump,FAKE AS REAL,0.0,0.47269,0.34723,165.04372,0.38835,316.0,1.0,FAM_FAKE_EYES,211.10852,156.49626
4,Yes,237.0,11.0,CELEBRITY_FAKE,Donald Trump,FAKE AS REAL,0.0,0.44957,0.34254,165.44803,1.06752,317.0,1.0,FAM_FAKE_EYES,44.74573,172.77437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1501,Yes,251.0,11.0,ALIKE_REAL,Barack Obama,REAL AS REAL,1.0,0.50491,0.62292,1270.06152,0.38818,2053.0,1.0,NaN,303.85880,267.47580
1502,Yes,251.0,11.0,ALIKE_REAL,Barack Obama,REAL AS REAL,1.0,0.48578,0.32849,1270.51440,0.38831,2054.0,1.0,ALIKE_REAL_MOUTH,355.22000,95.93495
1503,Yes,251.0,11.0,ALIKE_REAL,Barack Obama,REAL AS REAL,1.0,0.52169,0.67578,1270.96729,0.27502,2055.0,1.0,NaN,422.41284,279.39398
1504,Yes,251.0,11.0,ALIKE_REAL,Barack Obama,REAL AS REAL,1.0,0.56626,0.51420,1271.25854,0.19397,2056.0,1.0,NaN,211.94012,66.18616


,Response,VideoId,ParticipantId,VideoType,CelebrityName,Confusion,Correction,FPOGX,FPOGY,FPOGS,FPOGD,FPOGID,FPOGV,AOI,SACCADE_MAG,SACCADE_DIR
0,Yes,237.0,11.0,CELEBRITY_FAKE,Donald Trump,FAKE AS REAL,0.0,0.50185,0.28159,162.27788,0.29126,313.0,1.0,FAM_FAKE_EYES,451.72058,96.14632
1,Yes,237.0,11.0,CELEBRITY_FAKE,Donald Trump,FAKE AS REAL,0.0,0.51672,0.38556,162.58522,1.95715,314.0,1.0,FAM_FAKE_EYES-FAM_FAKE_MOUTH,127.98897,282.88937
2,Yes,237.0,11.0,CELEBRITY_FAKE,Donald Trump,FAKE AS REAL,0.0,0.57352,0.41739,164.55856,0.46890,315.0,1.0,NaN,115.55151,340.69760
3,Yes,237.0,11.0,CELEBRITY_FAKE,Donald Trump,FAKE AS REAL,0.0,0.47269,0.34723,165.04372,0.38835,316.0,1.0,FAM_FAKE_EYES,211.10852,156.49626
4,Yes,237.0,11.0,CELEBRITY_FAKE,Donald Trump,FAKE AS REAL,0.0,0.44957,0.34254,165.44803,1.06752,317.0,1.0,FAM_FAKE_EYES,44.74573,172.77437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1501,Yes,251.0,11.0,ALIKE_REAL,Barack Obama,REAL AS REAL,1.0,0.50491,0.62292,1270.06152,0.38818,2053.0,1.0,NaN,303.85880,267.47580
1502,Yes,251.0,11.0,ALIKE_REAL,Barack Obama,REAL AS REAL,1.0,0.48578,0.32849,1270.51440,0.38831,2054.0,1.0,ALIKE_REAL_MOUTH,355.22000,95.93495
1503,Yes,251.0,11.0,ALIKE_REAL,Barack Obama,REAL AS REAL,1.0,0.52169,0.67578,1270.96729,0.27502,2055.0,1.0,NaN,422.41284,279.39398
1504,Yes,251.0,11.0,ALIKE_REAL,Barack Obama,REAL AS REAL,1.0,0.56626,0.51420,1271.25854,0.19397,2056.0,1.0,NaN,211.94012,66.18616


,Response,VideoId,ParticipantId,VideoType,CelebrityName,Confusion,Correction,FPOGX,FPOGY,FPOGS,FPOGD,FPOGID,FPOGV,AOI,SACCADE_MAG,SACCADE_DIR


2001_0 is done!
